# RBE Core

> Core functions for Recursive Bayesian Estimation - probability utilities, Bayesian inference, particle filters, and visualization helpers

In [ ]:
#| default_exp rbe.core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Callable, Tuple, List, Union
from fastcore.test import test_eq, test_close
from fastcore.all import *

## Probability Utilities

Basic probability distribution functions following fast.ai style with short, clear names.

In [ ]:
#| export
def prob_normalize(probs):
    "Normalize `probs` to sum to 1"
    probs = np.asarray(probs)
    s = np.sum(probs)
    if s == 0: raise ValueError("Cannot normalize zero probabilities")
    return probs / s

def prob_sample(probs, n=1, rng=None):
    "Sample `n` indices from `probs` distribution"
    if rng is None: rng = np.random.default_rng()
    probs = np.asarray(probs)
    if np.any(probs < 0): raise ValueError("Probabilities must be non-negative")
    probs = prob_normalize(probs)  # Handle normalization and zero-sum check
    return rng.choice(len(probs), size=n, p=probs)

def prob_entropy(probs):
    "Calculate entropy of `probs` distribution"
    probs = prob_normalize(probs)
    probs = probs[probs > 0]  # Remove zeros to avoid log(0)
    return -np.sum(probs * np.log2(probs))

def prob_kl_div(p, q):
    "KL divergence from `q` to `p`"
    p, q = prob_normalize(p), prob_normalize(q)
    # Add small epsilon to avoid log(0)
    eps = 1e-10
    return np.sum(p * np.log((p + eps) / (q + eps)))

In [ ]:
# Test probability utilities
probs = [1, 2, 3]
normalized = prob_normalize(probs)
test_close(np.sum(normalized), 1.0)
test_close(normalized, [1/6, 2/6, 3/6])

# Test sampling
rng = np.random.default_rng(42)
samples = prob_sample([0.1, 0.7, 0.2], n=1000, rng=rng)
assert len(samples) == 1000
assert np.all((samples >= 0) & (samples <= 2))

# Test entropy
uniform = [0.5, 0.5]
certain = [1.0, 0.0]
assert prob_entropy(uniform) > prob_entropy(certain)

# Test KL divergence
p = [0.5, 0.5]
q = [0.5, 0.5]
test_close(prob_kl_div(p, q), 0.0, eps=1e-10)  # Same distributions

## Bayesian Core

Core Bayesian inference functions for updating beliefs with evidence.

In [ ]:
#| export
def bayes_update(prior, likelihood, evidence=None):
    "Update `prior` with `likelihood` and optional `evidence`"
    prior, likelihood = np.array(prior), np.array(likelihood)
    if evidence is None: evidence = (prior * likelihood).sum()
    if evidence == 0: raise ValueError("Impossible observation")
    return (prior * likelihood) / evidence

def bayes_sequential(priors, likelihoods, evidences=None):
    "Sequential updating of `priors` with `likelihoods` and optional `evidences`"
    if evidences is None:
        evidences = [None] * len(likelihoods)
    
    posterior = np.array(priors)
    posteriors = [posterior.copy()]
    
    for likelihood, evidence in zip(likelihoods, evidences):
        posterior = bayes_update(posterior, likelihood, evidence)
        posteriors.append(posterior.copy())
    
    return np.array(posteriors)

def bayes_posterior_predictive(posterior, likelihood_fn, n_samples=1000, rng=None):
    "Sample from posterior predictive distribution"
    if rng is None: rng = np.random.default_rng()
    
    # Sample parameter values from posterior
    param_samples = prob_sample(posterior, n_samples, rng)
    
    # Generate predictions for each parameter sample
    predictions = []
    for param_idx in param_samples:
        # likelihood_fn should return a distribution over observations
        obs_dist = likelihood_fn(param_idx)
        obs_sample = prob_sample(obs_dist, 1, rng)[0]
        predictions.append(obs_sample)
    
    return np.array(predictions)

In [ ]:
# Test Bayesian core functions
prior = np.array([0.3, 0.7])
likelihood = np.array([0.8, 0.2])
posterior = bayes_update(prior, likelihood)
test_close(np.sum(posterior), 1.0)
assert posterior[0] > prior[0]  # First hypothesis should increase

# Test sequential updating
priors = [0.5, 0.5]
likelihoods = [[0.9, 0.1], [0.8, 0.2], [0.7, 0.3]]
posteriors = bayes_sequential(priors, likelihoods)
assert posteriors.shape == (4, 2)  # Initial + 3 updates
test_close(np.sum(posteriors, axis=1), 1.0)  # All normalized

# Test posterior predictive (simple case)
posterior = [0.6, 0.4]
def simple_likelihood(param_idx):
    if param_idx == 0:
        return [0.8, 0.2]  # Biased toward observation 0
    else:
        return [0.3, 0.7]  # Biased toward observation 1

rng = np.random.default_rng(42)
predictions = bayes_posterior_predictive(posterior, simple_likelihood, n_samples=100, rng=rng)
assert len(predictions) == 100
assert np.all((predictions >= 0) & (predictions <= 1))

## Particle Filter Foundation

Core particle filter functions for Monte Carlo-based Bayesian inference.

In [ ]:
#| export
def pf_init(n_particles, state_dim, init_fn=None, rng=None):
    "Initialize particle filter with `n_particles` and `state_dim`"
    if rng is None: rng = np.random.default_rng()
    
    if init_fn is None:
        # Default: uniform initialization in [0, 1]
        particles = rng.uniform(0, 1, size=(n_particles, state_dim))
    else:
        particles = init_fn(n_particles, state_dim, rng)
    
    weights = np.ones(n_particles) / n_particles
    return particles, weights

def pf_predict(particles, weights, transition_fn, rng=None):
    "Prediction step: apply `transition_fn` to `particles`"
    if rng is None: rng = np.random.default_rng()
    
    new_particles = np.zeros_like(particles)
    for i, particle in enumerate(particles):
        new_particles[i] = transition_fn(particle, rng)
    
    return new_particles, weights  # Weights unchanged in prediction

def pf_update(particles, weights, observation, likelihood_fn):
    "Update step: weight `particles` using `observation` and `likelihood_fn`"
    new_weights = np.zeros_like(weights)
    
    for i, particle in enumerate(particles):
        new_weights[i] = weights[i] * likelihood_fn(particle, observation)
    
    # Normalize weights
    if np.sum(new_weights) > 0:
        new_weights = prob_normalize(new_weights)
    else:
        # If all weights are zero, reset to uniform
        new_weights = np.ones_like(weights) / len(weights)
    
    return particles, new_weights

def pf_resample(particles, weights, method='systematic', rng=None):
    "Resample `particles` using `weights` with specified `method`"
    if rng is None: rng = np.random.default_rng()
    n_particles = len(particles)
    
    if method == 'systematic':
        # Systematic resampling
        positions = (np.arange(n_particles) + rng.uniform()) / n_particles
        cum_weights = np.cumsum(weights)
        indices = np.searchsorted(cum_weights, positions)
    elif method == 'multinomial':
        # Multinomial resampling
        indices = prob_sample(weights, n_particles, rng)
    else:
        raise ValueError(f"Unknown resampling method: {method}")
    
    new_particles = particles[indices]
    new_weights = np.ones(n_particles) / n_particles
    
    return new_particles, new_weights

def pf_effective_size(weights):
    "Calculate effective sample size of normalized `weights`"
    weights = prob_normalize(weights)  # Ensure weights are normalized
    return 1.0 / np.sum(weights**2)

def pf_step(particles, weights, observation, transition_fn, likelihood_fn, 
           resample_threshold=0.5, rng=None):
    "Complete particle filter step: predict, update, and conditionally resample"
    # Prediction
    particles, weights = pf_predict(particles, weights, transition_fn, rng)
    
    # Update
    particles, weights = pf_update(particles, weights, observation, likelihood_fn)
    
    # Conditional resampling
    eff_size = pf_effective_size(weights)
    if eff_size < resample_threshold * len(particles):
        particles, weights = pf_resample(particles, weights, rng=rng)
    
    return particles, weights

In [ ]:
# Test particle filter functions
rng = np.random.default_rng(42)

# Test initialization
particles, weights = pf_init(100, 2, rng=rng)
assert particles.shape == (100, 2)
assert len(weights) == 100
test_close(np.sum(weights), 1.0)

# Test prediction with simple transition
def simple_transition(particle, rng):
    return particle + rng.normal(0, 0.1, size=particle.shape)

new_particles, new_weights = pf_predict(particles, weights, simple_transition, rng)
assert new_particles.shape == particles.shape
test_close(new_weights, weights)  # Weights unchanged

# Test update with simple likelihood
def simple_likelihood(particle, observation):
    # Gaussian likelihood
    diff = np.linalg.norm(particle - observation)
    return np.exp(-0.5 * diff**2)

observation = np.array([0.5, 0.5])
particles, weights = pf_update(particles, weights, observation, simple_likelihood)
test_close(np.sum(weights), 1.0)

# Test resampling
particles, weights = pf_resample(particles, weights, rng=rng)
test_close(np.sum(weights), 1.0)
test_close(weights, np.ones(100)/100)  # Should be uniform after resampling

# Test effective sample size
uniform_weights = np.ones(100) / 100
skewed_weights = np.zeros(100)
skewed_weights[0] = 1.0
assert pf_effective_size(uniform_weights) > pf_effective_size(skewed_weights)

## RBE Estimator

Main recursive Bayesian estimator implementations.

In [ ]:
#| export
def rbe_estimator(observations, transition_fn, likelihood_fn, 
                 n_particles=1000, init_fn=None, rng=None):
    "Main RBE estimator for `observations` with particle filter"
    if rng is None: rng = np.random.default_rng()
    
    # Infer state dimension from first observation
    state_dim = len(observations[0]) if hasattr(observations[0], '__len__') else 1
    
    # Initialize particles
    particles, weights = pf_init(n_particles, state_dim, init_fn, rng)
    
    # Store results
    estimates = []
    particle_history = [particles.copy()]
    weight_history = [weights.copy()]
    
    for obs in observations:
        # Particle filter step
        particles, weights = pf_step(particles, weights, obs, 
                                   transition_fn, likelihood_fn, rng=rng)
        
        # Estimate (weighted mean)
        estimate = np.average(particles, weights=weights, axis=0)
        estimates.append(estimate)
        
        # Store history
        particle_history.append(particles.copy())
        weight_history.append(weights.copy())
    
    return {
        'estimates': np.array(estimates),
        'particles': particle_history,
        'weights': weight_history
    }

def rbe_adaptive(observations, transition_fn, likelihood_fn, 
                adapt_rate=0.01, n_particles=1000, init_fn=None, rng=None):
    "Adaptive RBE estimator that adjusts parameters based on performance"
    if rng is None: rng = np.random.default_rng()
    
    # Start with base estimator
    result = rbe_estimator(observations, transition_fn, likelihood_fn, 
                          n_particles, init_fn, rng)
    
    # Simple adaptation: adjust resampling threshold based on effective size
    # This is a placeholder for more sophisticated adaptation
    avg_eff_size = np.mean([pf_effective_size(w) for w in result['weights']])
    adapted_threshold = max(0.1, min(0.9, 0.5 + adapt_rate * (avg_eff_size - n_particles/2)))
    
    result['adaptation_info'] = {
        'avg_effective_size': avg_eff_size,
        'adapted_threshold': adapted_threshold
    }
    
    return result

def rbe_metrics(true_states, estimates):
    "Calculate performance metrics for RBE estimates"
    true_states = np.array(true_states)
    estimates = np.array(estimates)
    
    # Mean squared error
    mse = np.mean((true_states - estimates)**2)
    
    # Root mean squared error
    rmse = np.sqrt(mse)
    
    # Mean absolute error
    mae = np.mean(np.abs(true_states - estimates))
    
    # Maximum absolute error
    max_ae = np.max(np.abs(true_states - estimates))
    
    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'max_absolute_error': max_ae,
        'n_samples': len(estimates)
    }

In [ ]:
# Test RBE estimator functions
rng = np.random.default_rng(42)

# Generate synthetic data
true_states = [np.array([i * 0.1, i * 0.05]) for i in range(10)]
observations = [state + rng.normal(0, 0.1, 2) for state in true_states]

# Define simple transition and likelihood
def test_transition(particle, rng):
    return particle + rng.normal(0, 0.05, particle.shape)

def test_likelihood(particle, observation):
    diff = np.linalg.norm(particle - observation)
    return np.exp(-0.5 * (diff / 0.1)**2)

# Test basic estimator
result = rbe_estimator(observations, test_transition, test_likelihood, 
                      n_particles=100, rng=rng)
assert result['estimates'].shape == (10, 2)
assert len(result['particles']) == 11  # Initial + 10 steps
assert len(result['weights']) == 11

# Test adaptive estimator
adaptive_result = rbe_adaptive(observations, test_transition, test_likelihood,
                              n_particles=100, rng=rng)
assert 'adaptation_info' in adaptive_result
assert 'avg_effective_size' in adaptive_result['adaptation_info']

# Test metrics
metrics = rbe_metrics(true_states, result['estimates'])
assert 'mse' in metrics
assert 'rmse' in metrics
assert 'mae' in metrics
assert metrics['n_samples'] == 10
assert metrics['rmse'] == np.sqrt(metrics['mse'])

## Visualization Helpers

Helper functions for visualizing RBE results and particle filters.

In [ ]:
#| export
def viz_particles(particles, weights, title='Particle Distribution', 
                 figsize=(8, 6), alpha=0.6):
    "Visualize `particles` with `weights`"
    fig, ax = plt.subplots(figsize=figsize)
    
    if particles.shape[1] == 1:
        # 1D case: histogram
        ax.hist(particles.flatten(), weights=weights, bins=30, alpha=alpha)
        ax.set_xlabel('State')
        ax.set_ylabel('Probability Density')
    elif particles.shape[1] == 2:
        # 2D case: scatter plot
        scatter = ax.scatter(particles[:, 0], particles[:, 1], 
                           s=weights*1000, alpha=alpha)
        ax.set_xlabel('State Dimension 1')
        ax.set_ylabel('State Dimension 2')
    else:
        # Higher dimensions: just show first two
        scatter = ax.scatter(particles[:, 0], particles[:, 1], 
                           s=weights*1000, alpha=alpha)
        ax.set_xlabel('State Dimension 1')
        ax.set_ylabel('State Dimension 2')
        title += f' (showing dims 1-2 of {particles.shape[1]})'
    
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    return fig, ax

def viz_beliefs(beliefs, time_steps=None, title='Belief Evolution', 
               figsize=(10, 6), labels=None):
    "Visualize evolution of `beliefs` over `time_steps`"
    beliefs = np.array(beliefs)
    if time_steps is None:
        time_steps = np.arange(len(beliefs))
    
    fig, ax = plt.subplots(figsize=figsize)
    
    if beliefs.ndim == 2:
        # Multiple belief dimensions
        for i in range(beliefs.shape[1]):
            label = f'Belief {i+1}' if labels is None else labels[i]
            ax.plot(time_steps, beliefs[:, i], label=label, marker='o')
        ax.legend()
    else:
        # Single belief dimension
        ax.plot(time_steps, beliefs, marker='o')
    
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Belief Value')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    return fig, ax

def viz_comparison(methods_data, time_steps=None, title='Method Comparison',
                  figsize=(12, 8), metrics=['mse', 'mae']):
    "Compare multiple methods with `methods_data` dict"
    if time_steps is None:
        # Try to infer time_steps from data
        first_method = list(methods_data.keys())[0]
        first_data = methods_data[first_method]
        
        # Look for estimates first, then fall back to any available metric
        if 'estimates' in first_data:
            time_steps = np.arange(len(first_data['estimates']))
        else:
            # Use the first available metric to infer length
            available_metrics = [m for m in metrics if m in first_data]
            if available_metrics:
                time_steps = np.arange(len(first_data[available_metrics[0]]))
            else:
                # Default fallback
                time_steps = np.arange(10)
    
    n_metrics = len(metrics)
    fig, axes = plt.subplots(n_metrics, 1, figsize=figsize)
    if n_metrics == 1:
        axes = [axes]
    
    for i, metric in enumerate(metrics):
        ax = axes[i]
        
        for method_name, method_data in methods_data.items():
            if metric in method_data:
                ax.plot(time_steps, method_data[metric], 
                       label=method_name, marker='o')
        
        ax.set_xlabel('Time Step')
        ax.set_ylabel(metric.upper())
        ax.set_title(f'{title} - {metric.upper()}')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig, axes

def viz_rbe_summary(rbe_result, true_states=None, title='RBE Summary',
                   figsize=(15, 10)):
    "Create comprehensive summary visualization of RBE results"
    fig = plt.figure(figsize=figsize)
    
    # Layout: 2x2 grid
    gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)
    
    # Top left: Final particle distribution
    ax1 = fig.add_subplot(gs[0, 0])
    final_particles = rbe_result['particles'][-1]
    final_weights = rbe_result['weights'][-1]
    
    if final_particles.shape[1] >= 2:
        ax1.scatter(final_particles[:, 0], final_particles[:, 1], 
                   s=final_weights*1000, alpha=0.6)
        ax1.set_xlabel('State Dim 1')
        ax1.set_ylabel('State Dim 2')
    else:
        ax1.hist(final_particles.flatten(), weights=final_weights, bins=30, alpha=0.6)
        ax1.set_xlabel('State')
        ax1.set_ylabel('Density')
    ax1.set_title('Final Particle Distribution')
    ax1.grid(True, alpha=0.3)
    
    # Top right: Estimates over time
    ax2 = fig.add_subplot(gs[0, 1])
    estimates = rbe_result['estimates']
    time_steps = np.arange(len(estimates))
    
    if estimates.ndim == 2 and estimates.shape[1] >= 2:
        ax2.plot(time_steps, estimates[:, 0], 'b-', label='Dim 1', marker='o')
        ax2.plot(time_steps, estimates[:, 1], 'r-', label='Dim 2', marker='s')
        if true_states is not None:
            true_states = np.array(true_states)
            ax2.plot(time_steps, true_states[:, 0], 'b--', alpha=0.7, label='True Dim 1')
            ax2.plot(time_steps, true_states[:, 1], 'r--', alpha=0.7, label='True Dim 2')
        ax2.legend()
    else:
        ax2.plot(time_steps, estimates.flatten(), 'b-', marker='o', label='Estimate')
        if true_states is not None:
            ax2.plot(time_steps, np.array(true_states).flatten(), 'r--', alpha=0.7, label='True')
        ax2.legend()
    
    ax2.set_xlabel('Time Step')
    ax2.set_ylabel('State Value')
    ax2.set_title('Estimates Over Time')
    ax2.grid(True, alpha=0.3)
    
    # Bottom left: Effective sample size
    ax3 = fig.add_subplot(gs[1, 0])
    eff_sizes = [pf_effective_size(w) for w in rbe_result['weights']]
    ax3.plot(np.arange(len(eff_sizes)), eff_sizes, 'g-', marker='o')
    ax3.axhline(len(rbe_result['weights'][0])/2, color='r', linestyle='--', alpha=0.7, label='N/2')
    ax3.set_xlabel('Time Step')
    ax3.set_ylabel('Effective Sample Size')
    ax3.set_title('Particle Filter Health')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Bottom right: Error metrics (if true states provided)
    ax4 = fig.add_subplot(gs[1, 1])
    if true_states is not None:
        errors = np.array(true_states) - estimates
        if errors.ndim == 2:
            error_norms = np.linalg.norm(errors, axis=1)
        else:
            error_norms = np.abs(errors)
        
        ax4.plot(time_steps, error_norms, 'r-', marker='o')
        ax4.set_xlabel('Time Step')
        ax4.set_ylabel('Estimation Error')
        ax4.set_title('Error Over Time')
    else:
        ax4.text(0.5, 0.5, 'No true states\nprovided', ha='center', va='center', 
                transform=ax4.transAxes, fontsize=12)
        ax4.set_title('Error Analysis')
    ax4.grid(True, alpha=0.3)
    
    fig.suptitle(title, fontsize=16)
    return fig

In [ ]:
# Test visualization functions (basic functionality)
rng = np.random.default_rng(42)

# Create test data
particles = rng.normal(0, 1, (100, 2))
weights = rng.exponential(1, 100)
weights = prob_normalize(weights)

# Test particle visualization
fig, ax = viz_particles(particles, weights)
assert fig is not None
plt.close(fig)

# Test belief visualization
beliefs = np.random.random((10, 3))
fig, ax = viz_beliefs(beliefs)
assert fig is not None
plt.close(fig)

# Test comparison visualization
methods_data = {
    'Method A': {
        'estimates': np.random.random(10), 
        'mse': np.random.random(10), 
        'mae': np.random.random(10)
    },
    'Method B': {
        'estimates': np.random.random(10), 
        'mse': np.random.random(10), 
        'mae': np.random.random(10)
    }
}
fig, axes = viz_comparison(methods_data)
assert fig is not None
plt.close(fig)

# Test RBE summary visualization
# Use the RBE result from previous test
true_states = [np.array([i * 0.1, i * 0.05]) for i in range(10)]
observations = [state + rng.normal(0, 0.1, 2) for state in true_states]
result = rbe_estimator(observations, test_transition, test_likelihood, 
                      n_particles=50, rng=rng)  # Smaller for faster test

fig = viz_rbe_summary(result, true_states)
assert fig is not None
plt.close(fig)

## Export Functions

Define all functions to be exported from this module.

In [ ]:
#| export
__all__ = [
    # Probability utilities
    'prob_normalize', 'prob_sample', 'prob_entropy', 'prob_kl_div',
    
    # Bayesian core
    'bayes_update', 'bayes_sequential', 'bayes_posterior_predictive',
    
    # Particle filter foundation
    'pf_init', 'pf_predict', 'pf_update', 'pf_resample', 'pf_effective_size', 'pf_step',
    
    # RBE estimator
    'rbe_estimator', 'rbe_adaptive', 'rbe_metrics',
    
    # Visualization helpers
    'viz_particles', 'viz_beliefs', 'viz_comparison', 'viz_rbe_summary'
]

## Summary

This module provides the complete foundation for Recursive Bayesian Estimation following fast.ai coding principles:

- **Short, clear names**: `prob_normalize`, `bayes_update`, `pf_init`
- **Function-first approach**: Minimal classes, comprehensive functions
- **Liberal imports**: Full `__all__` definition for `from rbe.core import *`
- **Inline testing**: Comprehensive tests using `test_eq` and `test_close`
- **Mathematical focus**: Functions optimized for interactive exploration
- **Modular design**: Mix-and-match functionality for different use cases

The module supports the entire RBE blog series with:
- Core probability and Bayesian inference functions
- Complete particle filter implementation
- Ready-to-use RBE estimators
- Comprehensive visualization tools
- Performance metrics and analysis functions